# Heuristics vs Mathematical Optimization Using the Transport Model

In [982]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import plotly.figure_factory as ff

## Data

#### Working seeds to obtain different OFVs
- 3 & 14
- 5 & 14

In [983]:
np.random.seed(5)
random.seed(14)
fake = Faker('en_US')
n_plants = 3
n_markets = 3

In [984]:
def generate_demands(markets):
    df = pd.DataFrame({
        'Markets': [fake.city() for n in range(markets)],
        'Demand': [random.randint(1, 8) * 100 for n in range(markets)]
    })
    return df

In [985]:
capacity = pd.DataFrame({
    'Canning Plants': [fake.city() for n in range(n_plants)],
    'Capacity': [random.randint(1, 10) * 100 for n in range(n_plants)]
})

demand = generate_demands(n_markets)
while capacity['Capacity'].sum() < demand['Demand'].sum():
    demand = generate_demands(n_markets)

distance = pd.DataFrame(
    (np.random.rand(n_plants, n_markets) * 10).round(1), 
    index=capacity['Canning Plants'], columns=demand['Markets'])

In [986]:
distance

Markets,Carrollhaven,Bowmanville,South Lori
Canning Plants,,,
Port Danielhaven,2.2,8.7,2.1
Lake Alexis,9.2,4.9,6.1
Brooksborough,7.7,5.2,3.0


In [987]:
# capacity = pd.DataFrame({
#     'Canning Plants': ['Seattle', 'San-Diego'],
#     'Capacity': [400, 500]
# })

# demand = pd.DataFrame({
#     'Markets': ['New-York', 'Chicago', 'Topeka'],
#     'Demand': [325, 300, 275]
# })

# distance = pd.DataFrame(np.array([[2.5, 1.7, 1.8],  [2.5, 1.8, 1.4]]), index=capacity['Canning Plants'], columns=demand['Markets'])

In [988]:
freight_cost = 90

transport_cost = freight_cost * distance / 1000

transport_cost

Markets,Carrollhaven,Bowmanville,South Lori
Canning Plants,,,
Port Danielhaven,0.198,0.783,0.189
Lake Alexis,0.828,0.441,0.549
Brooksborough,0.693,0.468,0.270


## Heuristic

- Sort transport cost from i to j in ascending order
- Select the combination of i and j with the lowest transportation cost
- Check if market j has unsatisfied demands 
- If j has unsatisfied demands, transport as much as possible from i to j 
- Select the next combination of i and j with lowest transportation cost
- Proceed until all demands are satisfied

In [989]:
total_cost = 0

sorted = transport_cost.unstack().sort_values(ascending=True)

current_demand = demand.set_index('Markets').to_dict()['Demand']
current_capacity = capacity.set_index('Canning Plants').to_dict()['Capacity']

for (j,i) in sorted.index:
    if current_demand[j] > 0:
        if current_capacity[i] >= current_demand[j]:
            ship = current_demand[j]
            current_demand[j] = 0
            current_capacity[i] -= ship
            total_cost += sorted[j,i] * ship
            print(f'Ship {ship} cases from {i} to {j} at {round(sorted[j,i] * ship, 2)} cost.')
        elif current_capacity[i] > 0:
            ship = current_capacity[i]
            current_capacity[i] = 0    
            current_demand[j] -= ship
            total_cost += sorted[j,i] * ship
            print(f'Ship {ship} cases from {i} to {j} at {round(sorted[j,i] * ship, 2)} cost.')
        # print(j, i, sorted[j,i])
# sorted
print('Leftover capacities', current_capacity)
print('Leftover demands', current_demand)
print('Total cost', total_cost)

Ship 200 cases from Port Danielhaven to South Lori at 37.8 cost.
Ship 300 cases from Brooksborough to South Lori at 81.0 cost.
Ship 500 cases from Lake Alexis to Bowmanville at 220.5 cost.
Ship 400 cases from Brooksborough to Carrollhaven at 277.2 cost.
Leftover capacities {'Port Danielhaven': 0, 'Lake Alexis': 500, 'Brooksborough': 200}
Leftover demands {'Carrollhaven': 0, 'Bowmanville': 0, 'South Lori': 0}
Total cost 616.5


## Mathematical Modeling

In [990]:
%reload_ext gams.magic

In [991]:
import gams.transfer as gt

In [992]:
%gams_reset
m = gams.exchange_container

# create sets
i = m.addSet('i', records=capacity['Canning Plants'], description='canning plants')
j = m.addSet('j', records=demand['Markets'], description='markets')

# create parameters
a = m.addParameter('a', [i], records=capacity, description='capacity of plant i in cases')
b = m.addParameter('b', [j], records=demand, description='demand at market j in cases')
c = m.addParameter('c', [i,j], records=transport_cost.unstack().reset_index()[['Canning Plants', 'Markets', 0]], description='transport cost in thousands of dollars per case')

# create variables
Z = m.addVariable('Z', description='total transportation costs in thousands of dollars')
X = m.addVariable('X', domain=[i,j], type='positive', description='shipment quantities in cases')

In [993]:
%%gams
Equations
     cost        define objective function
     supply(i)   observe supply limit at plant i
     demand(j)   satisfy demand at market j ;

cost ..        z  =e=  sum((i,j), c(i,j)*X(i,j)) ;
supply(i) ..   sum(j, X(i,j))  =l=  a(i) ;
demand(j) ..   sum(i, X(i,j))  =g=  b(j) ;

Model transport /all/ ;

Solve transport using LP minimizing z ;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),533.7,7,10,LP,CPLEX,0


In [994]:
opt_total_cost = Z.records['level'][0]

## Compare

In [1004]:
if round(total_cost, 1) != round(opt_total_cost, 1):
    print(f'Using the heuristic approach the total cost evaluates to {round(total_cost, 1)}.')
    print(f'Using the optimization approach the total cost evaluates to {round(opt_total_cost, 1)}.')
    print(f'Thus, optimization reduces the cost of transportation by {round((total_cost - opt_total_cost) / total_cost * 100, 2)} %.')

Using the heuristic approach the total cost evaluates to 616.5.
Using the optimization approach the total cost evaluates to 533.7.
Thus, optimization reduces the cost of transportation by 13.43 %.
